In [3]:
#我们实现含多个输入通道的互相关运算。我们只需要对每个通道做互相关运算，然后通过add_n函数进行累加
import torch
from torch import nn
import sys
sys.path.append("..")
import d2lzh_pytorch as d2l

def corr2d_multi_in(X,K):
    #沿着X和K的第0维分别计算再相加
    res = d2l.corr2d(X[0,:,:],K[0,:,:])
    for i in range(1,X.shape[0]):
        res += d2l.corr2d(X[i,:,:],K[i,:,:])
    return res

In [4]:
X = torch.tensor([[[0, 1, 2], [3, 4, 5], [6, 7, 8]],
              [[1, 2, 3], [4, 5, 6], [7, 8, 9]]])
K = torch.tensor([[[0, 1], [2, 3]], [[1, 2], [3, 4]]])

corr2d_multi_in(X, K)


tensor([[ 56.,  72.],
        [104., 120.]])

In [5]:
def corr2d_multi_in_out(X,K):
    #对k的第0维遍历，每次同输入X做互相关计算。所以结果使用stack函数合并在一起
    return torch.stack([corr2d_multi_in(X,k) for k in K])

#### 我们将核数组K同K+1(K中每个元素加1）和K+2联结在一起构造一个输出通道数为3的卷积核

In [7]:
K = torch.stack([K,K+1,K+2])
K.shape

torch.Size([3, 2, 2, 2])

#### 下面我们对输入数组X与核数组K做互相关运算。此时的输出含有3个通道。其中第一个通道的结果是与之前数组X与多输入通道，但输出通道核计算的结果一直

In [8]:
corr2d_multi_in_out(X,K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

#### 下面我们使用全连接层中的矩阵乘法来实现1*1卷积。这里需要在矩阵乘法运算前后对数据做一些调整

In [16]:
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.view(c_i, h * w)
    K = K.view(c_o, c_i)
    Y = torch.mm(K, X)  # 全连接层的矩阵乘法
    return Y.view(c_o, h, w)


In [17]:
X = torch.rand(3, 3, 3)
K = torch.rand(2, 3, 1, 1)

Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)

(Y1 - Y2).norm().item() < 1e-6


True